# **DeepZork**
***
Can a machine learn to play a text-based game?

<img src='zork_box_art.jpg'/>

## Project Motivation
***
Advances in machine learning involving reinforcement learning has seen the rise of AIs capable of playing video games and outperforming human players. These include all sorts of games from chess and Go to Atari games. All of these instances involve games in which gameplay is based on visuals. Using convolutional neural networks, machines become quite capable of recognizing patterns in changing pixels on a screen and can react in real-time in order to win. 

Enter text-based games.

Also known as interactive fiction games, text-based games were some of the first ever computer games due to relying solely on text for gameplay and not needing to process graphics. They rely on language parsing text commands given by the player in response to changing descriptions of the player's environment. 

This presents a whole different type of problem for AIs attempting to play one of these games. In text-based games, reaction time is not a factor. Instead, the player must have an understanding of the language used and to be able to make smart decisions based off of subtle hints in the game's descriptions.

For example, if a game tells the player that, to the north lies a monster that will kill the player if they head that direction, the player should know not to go that way.

There have been some attempts at teaching machines to play text-based games with a key example being __TextWorld__ by Microsoft with variable success. There is still, however, a long way to go before an interactive fiction AI will perform at the level of AlphaGo or AlphaZero.

This project will aide in tackling the issue by focusing on teaching a machine to play one single game - Zork - in hopes that the techniques used can be applied to other games in the future. 


## About Zork
***
### History
***
One of the most classic and well-known text-based game, Zork I (also known as Zork: The Great Underground Empire), was written in the late 1970s. At the time it was well-revered for its compelling story-telling and also its language parsing engines which allowed players to use not only simple commands ('Attack monster') but more complex ones with prepositions ('Attack the monster with the sword'). It was one of the most successful works of interactive fiction and its legacy is still alive today with many elements still being used in newer works

### Plot
***
Zork is first and foremost an adventure game. The ultimate goal of the game is to collect all 19 different treasures and install them in a trophy case. To do so, the player must explore all areas of the game, including a sprawling underground dungeon.

### Map
***
Below are two maps of the Zork world - above and below ground. As you can see it's a very large map with many different rooms and paths between them. Note that the player always starts at **West of House** on the above ground map.


<img src='zork_map_1.gif'/> <img src='zork_map_2.gif'/>

## Prework
***
### Commands
There are three types of commands used in Zork.

1) Basic commands - commands to go a certain direction, commands to get info
        - Go north, go south
        - Look, check inventory
2) Verb-Object commands - commands that consist of a verb and a noun or noun phrase
        - Take key, open the door
3) Verb-Object-Prep-Object commands - commands that consist of a verb and noun phrase followed by a preposition and second noun phrase
        - Attack the monster with the sword, unlock the chest with the key
    
The objects in these commands can be gathered from the descriptions in-game. The verbs however need to be provide beforehand.

### Verb dictionary

To generate a list of possible verbs used in commands, info from the game file itself was gathered using ZTools (https://www.inform-fiction.org/zmachine/ztools.html).
The text was processed using Matcher and POS from the <b>spacy</b> library to find all the possible verbs and verb phrases. This can be seen in the ZorkVerbs notebook.

This list was then manually picked through to remove commands that would not influence gameplay (commands that save the game, change the text descriptions, etc.)

Also, for the 2nd type of commands, the noun phrase was replaced with 'OBJ' and for the 3rd type of commands the first noun phrases were replaced with 'OBJ' and the second with 'DCT' to allow for easy substitution of nouns.

The commands were then combined into a <i>commands</i> class and stored in a seperate script file (game_commands.py). The lists can be seen below.

In [3]:
from game_commands import commands
cmd = commands()

In [11]:
## Basic commands
cmd.basic_actions

['go north',
 'go south',
 'go west',
 'go east',
 'go northeast',
 'go northwest',
 'go southeast',
 'go southwest',
 'd',
 'u']

In [7]:
## Verb-object commands
cmd.command1_actions

['open OBJ',
 'get OBJ',
 'set OBJ',
 'hit OBJ',
 'eat OBJ',
 'put OBJ',
 'cut OBJ',
 'dig OBJ',
 'ask OBJ',
 'fix OBJ',
 'make OBJ',
 'wear OBJ',
 'move OBJ',
 'kick OBJ',
 'kill OBJ',
 'find OBJ',
 'play OBJ',
 'feel OBJ',
 'hide OBJ',
 'read OBJ',
 'fill OBJ',
 'flip OBJ',
 'burn OBJ',
 'pick OBJ',
 'pour OBJ',
 'pull OBJ',
 'apply OBJ',
 'leave OBJ',
 'ask OBJ',
 'break OBJ',
 'enter OBJ',
 'curse OBJ',
 'shake OBJ',
 'burn OBJ',
 'inflate OBJ',
 'brandish OBJ',
 'donate OBJ',
 'squeeze OBJ',
 'attach OBJ',
 'find OBJ',
 'banish OBJ',
 'read OBJ',
 'enchant OBJ',
 'feel OBJ',
 'pour OBJ']

In [8]:
## Verb-object-prep-object commands
cmd.command2_actions

['pour OBJ on DCT',
 'hide OBJ in DCT',
 'pour OBJ in DCT',
 'move OBJ in DCT',
 'hide OBJ on DCT',
 'flip OBJ for DCT',
 'fix OBJ with DCT',
 'spray OBJ on DCT',
 'dig OBJ with DCT',
 'cut OBJ with DCT',
 'pick OBJ with DCT',
 'squeeze OBJ on DCT',
 'pour OBJ from DCT',
 'fill OBJ with DCT',
 'brandish OBJ at DCT',
 'burn OBJ with DCT',
 'flip OBJ with DCT',
 'read OBJ with DCT',
 'hide OBJ under DCT',
 'carry OBJ from DCT',
 'inflate OBJ with DCT',
 'unlock OBJ with DCT',
 'give OBJ to DCT',
 'carry OBJ to DCT',
 'spray OBJ with DCT']

### Command likelihood
***
When the nouns from the game's descriptions are substituted into the above commands, certain combinations might not make much sense. For example, if there is a door and a table in the environment, the command "cut the door with the key" would not make any sense.

To help with this problem, walkthroughs and tutorials from other text-based games can be studied to have a way to determine the relevance and likelihood of different commands.

This can all be found in the scrape_tutorials notebook but the basic idea is that tutorials and walkthroughs from two databases - https://www.ifarchive.org/indexes/if-archive/solutions/ and http://www.textfiles.com/adventure/ were scraped using __urllib__ and __BeautifulSoup__ to get text files containing relevant commands for these games.

Those text files were then clean, preprocessed, and combined into one single file. The entire corpus was then ran through NLP using __spacy__.

To determine likelihood of command phrases, a Word2Vec model was used to determine the similarity score.

An example of this usage can be seen below.

In [13]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
f = open('tutorials_2.txt', 'r')
tutorials = f.read()
sentences = word_tokenize(tutorials)
w2v = Word2Vec([sentences])

In [56]:
w2v.wv.similarity('open', 'table')

0.1603776

In [57]:
w2v.wv.similarity('open', 'chest')

0.97911215

In [58]:
w2v.wv.similarity(word_tokenize('break chest with'), 'key').mean()

0.39725515

In [59]:
w2v.wv.similarity(word_tokenize('unlock chest with'), 'key').mean()

0.4157865